In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from SimCLR_data_util import preprocess_for_train
from SimCLR import *
from resnet_small import ResNet18
from tensorflow.keras.layers import Dense
from viewmaker import *
from logistic_regression import *

In [2]:
from datasets.cifar_10 import get_unsupervised_dataset
dataset = get_unsupervised_dataset(batch_size=64) 

In [3]:
augmentation = Viewmaker()

In [4]:
def get_projection_head():

    projection_head = tf.keras.Sequential([
        Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(256)
    ])

    return projection_head

projection_head = get_projection_head()

In [5]:
def get_encoder():
    model = ResNet18(10)
    encoder = tf.keras.Sequential(model.layers[:-1])
    return encoder

encoder = get_encoder()

In [6]:
viewmaker_model = SimCLR(encoder, augmentation, projection_head, temperature=0.07)

In [7]:
viewmaker_model.load_weights('./cifar_10_experiment/viewmaker_and_simclr_weights/ckpt1')
encoder = viewmaker_model.encoder

In [8]:
from datasets.cifar_10 import get_supervised_dataset
dataset = get_supervised_dataset()     

In [9]:
classifier = TrainClassifier(10, encoder)

In [11]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                    loss=tf.keras.losses.categorical_crossentropy,
                    metrics=[tf.keras.metrics.CategoricalAccuracy()])
x = tf.convert_to_tensor(dataset[0][0])
y = dataset[0][1]
x_val = tf.convert_to_tensor(dataset[1][0][:5000])
y_val = dataset[1][1][:5000]
classifier.fit(x=x, y=y, validation_data=(x_val, y_val), batch_size=512, epochs=500)

Epoch 1/500
98/98 [==============================] - 10s 94ms/step - loss: 0.4678 - categorical_accuracy: 0.8378 - val_loss: 0.8151 - val_categorical_accuracy: 0.7310
Epoch 2/500
98/98 [==============================] - 8s 81ms/step - loss: 0.3624 - categorical_accuracy: 0.8802 - val_loss: 0.8546 - val_categorical_accuracy: 0.7254
Epoch 3/500
98/98 [==============================] - 8s 80ms/step - loss: 0.3109 - categorical_accuracy: 0.9006 - val_loss: 0.8873 - val_categorical_accuracy: 0.7246
Epoch 4/500
98/98 [==============================] - 8s 80ms/step - loss: 0.2701 - categorical_accuracy: 0.9190 - val_loss: 0.9209 - val_categorical_accuracy: 0.7208
Epoch 5/500
98/98 [==============================] - 8s 81ms/step - loss: 0.2400 - categorical_accuracy: 0.9306 - val_loss: 0.9566 - val_categorical_accuracy: 0.7158
Epoch 6/500
98/98 [==============================] - 8s 80ms/step - loss: 0.2135 - categorical_accuracy: 0.9411 - val_loss: 0.9934 - val_categorical_accuracy: 0.7172
Epo

KeyboardInterrupt: 